In [ ]:
import sympy as sp
sp.init_printing()

import IPython.display as disp

# 1D
# Continuous piecewise linear functions
drawing = f"""
    .____.________.
    x-1  x0      x1
    |____|________|
      hx-    hx+

"""
print(drawing)

In [ ]:
# !!! Indicates a hypothesis (should be studied and considered)

# !!! Mode selection
mode_values = "real" # real or complex values


In [ ]:
# Wavenumbers
k = sp.Symbol('k',
              #real=True, # !!!
              #positive=True, # !!!
              )
kh = sp.symbols('k^h')

# 1D variable
x = sp.symbols('x')

# 1D variational formulations over [a, b]
tau = sp.symbols('tau')
def a_G(u, v, a, b):
    D_u = sp.diff(u, x)
    D_v = sp.diff(v, x)
    return sp.integrate(D_u*D_v, (x, a, b)) - ( k**2 * sp.integrate(u*v, (x, a, b)) )
def L(u):
    return sp.diff(u, x, 2) + ( k**2 * u )
def a_GLS(u, v, a, b):
    return a_G(u, v, a, b) + ( tau * sp.integrate(L(u)*L(v), (x, a, b)) )


In [ ]:

# h definitions
h_x_ = dict()
h_x_["-"], h_x_["+"] = sp.symbols('h_x- h_x+')
h_x = sp.symbols('h_x')

# Grid sample
x_ = dict()
x_["-1"], x_["0"], x_["+1"] = -h_x_["-"], sp.sympify(0), +h_x_["+"]

# Shape functions
N_ = dict()
N_["-1"] = - x / h_x_["-"]
N_["0-"] = (x / h_x_["-"]) + 1
N_["0+"] = (- x / h_x_["+"]) + 1
N_["+1"] = x / h_x_["+"]


In [ ]:
# Numerical solution is supposedly
if mode_values == "complex":
    uh = sp.exp(1j * kh * x)
elif mode_values == "real":
    uh = sp.cos(kh * x)
Uh_ = dict()
Uh_["-1"] = uh.subs(x, x_["-1"])
Uh_["0"] = uh.subs(x, x_["0"])
Uh_["+1"] = uh.subs(x, x_["+1"])

In [ ]:
# Linear system coefficients (GLS)
A_GLS_ = dict()
A_GLS_["0, -1"] = a_GLS(N_["0-"], N_["-1"], x_["-1"], x_["0"])
A_GLS_["0, 0"] = a_GLS(N_["0-"], N_["0-"], x_["-1"], x_["0"]) + a_GLS(N_["0+"], N_["0+"], x_["0"], x_["+1"])
A_GLS_["0, +1"] = a_GLS(N_["0+"], N_["+1"], x_["0"], x_["+1"])
print("GLS coefficients")
for a in A_GLS_.values():
    disp.display(a)

In [ ]:
# Construct the GLS Stencil Equation
stencil_eq = (A_GLS_["0, -1"] * Uh_["-1"]) + \
             (A_GLS_["0, 0"]  * Uh_["0"]) + \
             (A_GLS_["0, +1"] * Uh_["+1"])

In [ ]:
# Match k and k^h
stencil_eq_ideal = stencil_eq.subs(kh, k)

In [ ]:
# Solve for tau directly
tau_solutions = sp.solve(stencil_eq_ideal, tau)

print(f"Number of tau solutions found: {len(tau_solutions)}")
tau_opt = tau_solutions[0] # 1 tau

print("Optimal tau (general):")
disp.display(tau_opt)

In [ ]:
# Simplify and Check Consistency for Uniform Grid (hx+ = hx- = h)
print("\nChecking consistency for uniform grid (hx+ = hx- = h)...")
tau_opt_uniform = tau_opt.subs(h_x_["+"], h_x).subs(h_x_["-"], h_x)
tau_opt_uniform = sp.simplify(tau_opt_uniform)

print("Optimal tau (uniform):")
disp.display(tau_opt_uniform)